In [ ]:
import pickle as pkl
import pandas as pd

In [ ]:
with open("./training_data_pandas.pkl", "br") as fh:
    data = pkl.load(fh)

In [ ]:
# TASK: some data exloration

# what kind object is "data"?
# what columns are there?
# what is the list of unique station names?

# rsds_stations is the in-situ observations in W/m**2
# integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time_nora3 is the nora3 estimate
# and surface_solar_radiation_downwards_era5 the era5 one
# these are in Wh/m**2 (accumulated hourly values); create rsds_era5 and rsds_nora3 columns in data with the same normalization as rsds_stations

# plot rsds_stations for 1 station; what can you observe?
# what is the distribution of data in the rsds_stations column for the station? Is this a well balanced dataset? What could you do to make it "better balanced"?

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data.name.unique()

In [ ]:
data["rsds_nora3"] = data["integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time_nora3"] / 3600.0

In [ ]:
data["rsds_era5"] = data["surface_solar_radiation_downwards_era5"] / 3600.0

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# %matplotlib widget
%matplotlib tk

crrt_data = data[data["name"] == "OSLO - BLINDERN"]

plt.figure()
plt.plot(crrt_data["time_stations"], crrt_data["rsds_stations"])
plt.ylabel("W/m**2")
plt.show()

In [ ]:
# %matplotlib widget
%matplotlib tk

crrt_data = data[data["name"] == "TRONDHEIM - GLØSHAUGEN"]

plt.figure()
plt.plot(crrt_data["time_stations"], crrt_data["rsds_stations"])
plt.ylabel("W/m")

In [ ]:
%matplotlib tk

crrt_data = data[data["name"] == "TRONDHEIM - GLØSHAUGEN"]

plt.figure()
plt.hist(crrt_data["rsds_stations"], bins=300)
plt.show()

In [ ]:
##################################################################################################################

In [ ]:
# TASK: prepare the data

# Keep only the data where:
# “full_valid_flag” is True
# rsds_stations > 50.0
# rsds_nora3 > 50.0

# split between the training and validation datasets; use as a validation dataset the specific stations:
# "DOVRE-LANNEM",
# "TJØLLING",
# "GRAN",
# "RAKKESTAD",
# "ÅS",
# make *really* sure that the stations are well separated between the two, this is really important!

In [ ]:
import numpy as np

In [ ]:
valid_indexes = np.logical_and.reduce(
    (
        data["full_valid_flag"] == True,
        data["rsds_stations"] > 50.0,  # only use the cases when there is actually sun energy (not night)
        data["rsds_nora3"] > 50.0,  # only use the cases when there is actually sun energy (not night)
    )
)

In [ ]:
valid_data = data.loc[valid_indexes].reset_index()
print(f"size of valid_pd_all_data: {len(valid_data):,}")

In [ ]:
list_stations_for_validation = [
    "DOVRE-LANNEM",
    "TJØLLING",
    "GRAN",
    "RAKKESTAD",
    "ÅS",
]

In [ ]:
list_name_match_validation = [valid_data["name"] == st for st in list_stations_for_validation]
tuple_name_match_validation = tuple(list_name_match_validation)

# list our conditions to be a validation station
valid_indexes_validation = np.logical_or.reduce(
    tuple_name_match_validation
)

data_validation = valid_data.loc[valid_indexes_validation].reset_index().squeeze()
data_training = valid_data.loc[np.logical_not(valid_indexes_validation)].reset_index().squeeze()

In [ ]:
print(f"{len(data_validation) = }")
print(f"{len(data_training) = }")
print(f"{len(data_validation) + len(data_training) = }")

In [ ]:
data_validation.name.unique()

In [ ]:
data_training.name.unique()

In [ ]:
##################################################################################################################

In [ ]:
# TASK

# print the MAE, RMSE, relative variance level, between the rsds_nora3, rsds_era5, SSI_value_cmsaf_sis
# and the rsds_stations

# do this for both the full dataset, and for the training and validation datasets individually

# plot and interpret the data in a Taylor plot

In [ ]:
def print_stats(data_in, estimate_name):
    rsds = data_in["rsds_stations"].to_numpy()
    estimate = data_in[estimate_name].to_numpy()
    
    mae = np.mean(np.abs(rsds-estimate))
    rmse = np.sqrt(np.mean((rsds-estimate)**2))
    variance_ratio = np.std(estimate) / np.std(rsds)

    print(f"{estimate_name = }")
    print(f"{mae = }")
    print(f"{rmse = }")
    print(f"{variance_ratio = }")
    print("")

In [ ]:
data_stats = valid_data

print("all valid data")
print("")

print_stats(data_stats, "rsds_nora3")
print_stats(data_stats, "rsds_era5")
print_stats(data_stats, "SSI_value_cmsaf_sis")

In [ ]:
data_stats = data_training

print("data_training")
print("")

print_stats(data_stats, "rsds_nora3")
print_stats(data_stats, "rsds_era5")
print_stats(data_stats, "SSI_value_cmsaf_sis")

In [ ]:
data_stats = data_validation

print("data_validation")
print("")

print_stats(data_stats, "rsds_nora3")
print_stats(data_stats, "rsds_era5")
print_stats(data_stats, "SSI_value_cmsaf_sis")

In [ ]:
import skill_metrics as sm
from skill_metrics.check_taylor_stats import check_taylor_stats
import time

In [ ]:
def get_taylor_plot_stats(data_in, field_label_name, field_model_name):
    field_label_np = data_in.loc[:, field_label_name].to_numpy().astype(np.float32)
    field_model_np = data_in.loc[:, field_model_name].to_numpy().astype(np.float32)
    
    sdev_ref_label = np.std(field_label_np)
    field_label_np_normalized = field_label_np / sdev_ref_label
    field_model_np_normalized = field_model_np / sdev_ref_label

    sdev = np.std(field_model_np_normalized)
    crmsd = sm.centered_rms_dev(field_label_np_normalized, field_model_np_normalized)
    ccoef = np.corrcoef(field_label_np_normalized, field_model_np_normalized)[0,1]
    
    return (sdev, crmsd, ccoef)

In [ ]:
plt.rc('font', size=17)

plt.figure(num=1, figsize=(16, 12))

first_Taylor_diagram = True

list_colors = ["r", "k", "y", "b", "g", "m", "c", "darkorange", "lime", "paleturquoise", "gold"]
dict_label = {"observations": "c"}

data_to_use = data_validation

init_sdev, init_crmsd, init_ccoef = get_taylor_plot_stats(data_to_use, "rsds_stations", "rsds_stations")

list_columns_model = [
    "rsds_nora3",
    "rsds_era5",
    "SSI_value_cmsaf_sis",
]

for crrt_color, crrt_model in zip(list_colors[0:len(list_columns_model)], list_columns_model):
    dict_label[crrt_model] = crrt_color
    
    crrt_list_sdev = [init_sdev]
    crrt_list_crmsd = [init_crmsd]
    crrt_list_ccoef = [init_ccoef]
    
    list_stations_to_use = data_to_use.name.unique()
    
    for crrt_station in list_stations_to_use:
        
        pd_crrt_station = data_to_use[data_to_use["name"] == crrt_station]

        crrt_sdev, crrt_crmsd, crrt_ccoef = get_taylor_plot_stats(pd_crrt_station, "rsds_stations", crrt_model)
        crrt_list_sdev.append(crrt_sdev)
        crrt_list_crmsd.append(crrt_crmsd)
        crrt_list_ccoef.append(crrt_ccoef)
        
    crrt_sdev = np.array(crrt_list_sdev)
    crrt_crmsd = np.array(crrt_list_crmsd)
    crrt_ccoef = np.array(crrt_list_ccoef)
    
    if first_Taylor_diagram:
        sm.taylor_diagram(crrt_sdev, crrt_crmsd, crrt_ccoef, markercolor=crrt_color, markersize=4)
    else:
        sm.taylor_diagram(crrt_sdev, crrt_crmsd, crrt_ccoef, overlay = 'on', markercolor=crrt_color, markerLabel=dict_label, markersize=5)
    
    first_Taylor_diagram = False
    
    pd_to_aggregate = pd.DataFrame()
    
    for crrt_station in list_stations_to_use:
        pd_crrt_station = data_to_use[data_to_use["name"] == crrt_station]
        if len(pd_crrt_station) < 20:
            continue
        pd_to_aggregate = pd.concat([pd_to_aggregate, pd_crrt_station], ignore_index=True)
    
    # inspired from https://github.com/PeterRochford/SkillMetrics/blob/master/skill_metrics/taylor_diagram.py to put in the correct polar coordinates
    crrt_sdev, crrt_crmsd, crrt_ccoef = get_taylor_plot_stats(pd_to_aggregate, "rsds_stations", crrt_model)
    crrt_sdev = np.array([crrt_sdev])
    crrt_crmsd = np.array([crrt_crmsd])
    crrt_ccoef = np.array([crrt_ccoef])
    check_taylor_stats(crrt_sdev, crrt_crmsd, crrt_ccoef, 0.01)
    rho, theta = crrt_sdev, np.arccos(crrt_ccoef)
    
    plt.scatter(rho * np.cos(theta), rho * np.sin(theta), s=[512], c=[crrt_color], marker="X", label=crrt_model)
    
plt.scatter([1.0], [0.0], s=[512], c=["c"], label="truth")
plt.legend(bbox_to_anchor=(1.25, 1.25), loc='upper right')
plt.tight_layout()

In [ ]:
plt.rc('font', size=17)

plt.figure(num=1, figsize=(16, 12))

first_Taylor_diagram = True

list_colors = ["r", "k", "y", "b", "g", "m", "c", "darkorange", "lime", "paleturquoise", "gold"]
dict_label = {"observations": "c"}

data_to_use = data_training

init_sdev, init_crmsd, init_ccoef = get_taylor_plot_stats(data_to_use, "rsds_stations", "rsds_stations")

list_columns_model = [
    "rsds_nora3",
    "rsds_era5",
    "SSI_value_cmsaf_sis",
]

for crrt_color, crrt_model in zip(list_colors[0:len(list_columns_model)], list_columns_model):
    dict_label[crrt_model] = crrt_color
    
    crrt_list_sdev = [init_sdev]
    crrt_list_crmsd = [init_crmsd]
    crrt_list_ccoef = [init_ccoef]
    
    list_stations_to_use = data_to_use.name.unique()
    
    for crrt_station in list_stations_to_use:
        
        pd_crrt_station = data_to_use[data_to_use["name"] == crrt_station]

        crrt_sdev, crrt_crmsd, crrt_ccoef = get_taylor_plot_stats(pd_crrt_station, "rsds_stations", crrt_model)
        crrt_list_sdev.append(crrt_sdev)
        crrt_list_crmsd.append(crrt_crmsd)
        crrt_list_ccoef.append(crrt_ccoef)
        
    crrt_sdev = np.array(crrt_list_sdev)
    crrt_crmsd = np.array(crrt_list_crmsd)
    crrt_ccoef = np.array(crrt_list_ccoef)
    
    if first_Taylor_diagram:
        sm.taylor_diagram(crrt_sdev, crrt_crmsd, crrt_ccoef, markercolor=crrt_color, markersize=4)
    else:
        sm.taylor_diagram(crrt_sdev, crrt_crmsd, crrt_ccoef, overlay = 'on', markercolor=crrt_color, markerLabel=dict_label, markersize=5)
    
    first_Taylor_diagram = False
    
    pd_to_aggregate = pd.DataFrame()
    
    for crrt_station in list_stations_to_use:
        pd_crrt_station = data_to_use[data_to_use["name"] == crrt_station]
        if len(pd_crrt_station) < 20:
            continue
        pd_to_aggregate = pd.concat([pd_to_aggregate, pd_crrt_station], ignore_index=True)
    
    # inspired from https://github.com/PeterRochford/SkillMetrics/blob/master/skill_metrics/taylor_diagram.py to put in the correct polar coordinates
    crrt_sdev, crrt_crmsd, crrt_ccoef = get_taylor_plot_stats(pd_to_aggregate, "rsds_stations", crrt_model)
    crrt_sdev = np.array([crrt_sdev])
    crrt_crmsd = np.array([crrt_crmsd])
    crrt_ccoef = np.array([crrt_ccoef])
    check_taylor_stats(crrt_sdev, crrt_crmsd, crrt_ccoef, 0.01)
    rho, theta = crrt_sdev, np.arccos(crrt_ccoef)
    
    plt.scatter(rho * np.cos(theta), rho * np.sin(theta), s=[512], c=[crrt_color], marker="X", label=crrt_model)
    
plt.scatter([1.0], [0.0], s=[512], c=["c"], label="truth")
plt.legend(bbox_to_anchor=(1.25, 1.25), loc='upper right')
plt.tight_layout()

In [ ]:
##################################################################################################################

In [ ]:
# TASK: Apply a simple linear regression model

# The simplest model one can think about is a linear regression model
# Apply a linear regression model to the problem; choose the predictors that make sense
# How do the results look like? How does this relate to the “BLUE” estimator theory?
# “Why” does this work?
# Can you estimate how much importance is given to each predictor?

In [ ]:
import scipy as sp

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.compose import TransformedTargetRegressor

In [ ]:
label_column = "rsds_stations"

list_predictors_to_use = [
    'integral_of_surface_net_downward_shortwave_flux_wrt_time_nora3',
    'integral_of_toa_net_downward_shortwave_flux_wrt_time_nora3',
    'air_temperature_2m_nora3',
    'low_type_cloud_area_fraction_nora3',
    'cloud_area_fraction_nora3',
    'lwe_thickness_of_atmosphere_mass_content_of_water_vapor_nora3',
    'convective_cloud_area_fraction_nora3',
    'medium_type_cloud_area_fraction_nora3',
    'high_type_cloud_area_fraction_nora3',
    'precipitation_amount_acc_nora3',
    'rsds_nora3',
    'rsds_era5',
    'snowfall_amount_acc_nora3',
    'ASN_VEG_nora3',
    'TALB_ISBA_nora3',
    'LAI_nora3',
    'VEG_nora3',
    'top_net_solar_radiation_era5',
    'surface_net_solar_radiation_era5',
    'surface_solar_radiation_downwards_era5',
    'total_cloud_cover_era5',
    'high_cloud_cover_era5',
    'medium_cloud_cover_era5',
    'low_cloud_cover_era5',
    'snowfall_era5',
    'total_precipitation_era5',
    'snow_albedo_era5',
    'forecast_albedo_era5',
    'leaf_area_index_low_vegetation_era5',
    'leaf_area_index_high_vegetation_era5',
    'total_column_water_vapour_era5',
    'SSI_value_cmsaf_sis_sarah',
    'SSI_value_cmsaf_sis_sarah_minus_30mins',
    'SSI_value_cmsaf_sis_sarah_plus_30mins',
]

for crrt_predictor in list_predictors_to_use:
    if crrt_predictor not in data:
        raise RuntimeError(f"predictor {crrt_predictor} is required but not found in data")

In [ ]:
# ------------------------------
# extract: labels and predictors extraction

np_predictors_training = data_training.loc[:, list_predictors_to_use].to_numpy(dtype=np.float32)
np_labels_training = data_training.loc[:, label_column].to_numpy(dtype=np.float32).squeeze()
np_rsds_stations_training = data_training.loc[:, "rsds_stations"].to_numpy(dtype=np.float32).squeeze()

np_predictors_validation = data_validation.loc[:, list_predictors_to_use].to_numpy(dtype=np.float32)
np_labels_validation = data_validation.loc[:, label_column].to_numpy(dtype=np.float32).squeeze()
np_rsds_stations_validation = data_validation.loc[:, "rsds_stations"].to_numpy(dtype=np.float32).squeeze()

In [ ]:
# perform the linear fitting and prediction

X_train = np_predictors_training
y_train = np_labels_training
X_test = np_predictors_validation
y_test = np_labels_validation

# Create the linear regression object
# "vanilla" linear regression
model_kind = "LR"
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict the target variable using the test data
y_pred = model.predict(X_test)

In [ ]:
# perform the predictions
linear_predicted_training = np.array(model.predict(X_train)).squeeze()
linear_predicted_validation = np.array(model.predict(X_test)).squeeze()

data_training["rsds_sklearn"] = linear_predicted_training
data_validation["rsds_sklearn"] = linear_predicted_validation

# show statistics
print("sklearn linear regression")

# on training data
print("--- training ---")
print("linear model")
print(f"{np.mean(np.abs((linear_predicted_training-np_rsds_stations_training))) = }")
print(f"{np.sqrt(np.mean((linear_predicted_training-np_rsds_stations_training)**2)) = }")

# on validation data
print("--- validation ---")
print("linear model")
print(f"{np.mean(np.abs((linear_predicted_validation-np_rsds_stations_validation))) = }")
print(f"{np.sqrt(np.mean((linear_predicted_validation-np_rsds_stations_validation)**2)) = }")

print("")

In [ ]:
list_predictors_to_use_plot = list_predictors_to_use

In [ ]:
import seaborn

In [ ]:
# look at coeff * std, to see the importance of each coeff in the model (not this is only "marginal" importance when keeping everything else the same

coefs = pd.DataFrame(
    model.coef_ * X_train.std(axis=0) / y_train.std(axis=0),
    columns=["Coefficient importance"],
    index=list_predictors_to_use,
)

coefs_to_plot = coefs[coefs.index.isin(list_predictors_to_use_plot)]
print(f"{len(coefs_to_plot)}")

coefs_to_plot.plot(kind="barh", figsize=(9, 7))
plt.xlabel("Coefficient values corrected by the feature's std. dev.")
plt.title("")
plt.axvline(x=0, color=".5")
plt.subplots_adjust(left=0.3)
plt.title("Sklean linear model")

import seaborn as sns
# Declaring the cm variable by the
# color palette from seaborn
cm = sns.light_palette("green", as_cmap=True)

coefs.style.background_gradient(cmap=cm)

In [ ]:
##################################################################################################################

In [ ]:
# TASK: training a simple model

# What kind of NN model would make sense in the present situation

# Define and train the NN model in Keras
# How do the results compare with the other methods?
# Put it all in a common Taylor plot

In [ ]:
import tensorflow
import keras
from tensorflow.keras import layers

In [ ]:
print("perform normalization adaptation")

labels_inv_normalization_layer = layers.Normalization(invert=True, input_shape=[1,], axis=None)
labels_inv_normalization_layer.adapt(np_labels_training)

predictors_normalization_layer = layers.Normalization()
predictors_normalization_layer.adapt(np_predictors_training)

print("done")

In [ ]:
# ML setup

# define a simple NN

input_layer = keras.layers.Input(shape=(np_predictors_training.shape[1], ))
normalized_input = predictors_normalization_layer(input_layer)

fully_connected_1 = keras.layers.Dense(40, activation="relu")(normalized_input)
fully_connected_2 = keras.layers.Dense(40, activation="relu")(fully_connected_1)
fully_connected_3 = keras.layers.Dense(40, activation="relu")(fully_connected_2)
fully_connected_4 = keras.layers.Dense(40, activation="relu")(fully_connected_3)

output = keras.layers.Dense(1)(fully_connected_4)
denormalized_output = labels_inv_normalization_layer(output)

# metaparameters
learning_rate = 5e-4
loss_kind = "mean_absolute_error"

# model
keras_model = keras.Model(inputs=input_layer, outputs=denormalized_output)
keras_model.compile(optimizer=keras.optimizers.Adagrad(learning_rate=learning_rate), loss=loss_kind)
keras_model.summary()

In [ ]:
epochs = int(4e1)
min_delta_stop = 1e-7
patience_stop = 10

# training config
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=min_delta_stop, patience=patience_stop)

# shuffling of the data
permutation_indexes = np.random.permutation(np.arange(0, np_predictors_training.shape[0]))
np_predictors_training_perm = np_predictors_training[permutation_indexes, :]
np_labels_training_perm = np_labels_training[permutation_indexes]

# reduce the size of the dataset for going faster in tests
reduce_dataset = False
reduction_keep_factor = 0.3  # only use 1/3 of the data

if reduce_dataset:
    print("W reducing the size of the dataset;")
    print("W this is only fine for testing")
    max_index = int(reduction_keep_factor * np_predictors_training.shape[0])
else:
    max_index = np_predictors_training.shape[0]
    
np_predictors_training_perm_red = np_predictors_training_perm[:max_index, :]
np_labels_training_perm_red = np_labels_training_perm[:max_index]

# perform learning
history = keras_model.fit(
    x=np_predictors_training_perm_red,
    y=np_labels_training_perm_red,
    epochs=epochs,
    validation_split=0.10,
    # what callbacks to use... seems the checkpoint callback does not help and only slows down
    # callbacks=[es_callback, model_ckpt_callback, scheduler_callback]
    callbacks=[es_callback],
)

print()
print("done!")

In [ ]:
# learning curve
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epoch = list(range(len(loss)))

# to avoid obscuring the learning curve with initial sharp decay
start_epoch_idx = 10

plt.figure()
plt.plot(epoch[start_epoch_idx:], loss[start_epoch_idx:], "b", label="training loss")
plt.plot(epoch[start_epoch_idx:], val_loss[start_epoch_idx:], "k", label="validation loss")
plt.title("training curve")
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

In [ ]:
# perform the predictions
predicted_training = np.array(keras_model(np_predictors_training)).squeeze()
predicted_validation = np.array(keras_model(np_predictors_validation)).squeeze()

# on training data
print("--- training ---")
print("nn")
print(f"{np.mean(np.abs((predicted_training-np_rsds_stations_training))) = }")
print(f"{np.sqrt(np.mean((predicted_training-np_rsds_stations_training)**2)) = }")

# on validation data
print("--- validation ---")
print("nn")
print(f"{np.mean(np.abs((predicted_validation-np_rsds_stations_validation))) = }")
print(f"{np.sqrt(np.mean((predicted_validation-np_rsds_stations_validation)**2)) = }")

data_training["rsds_fcnn"] = predicted_training
data_validation["rsds_fcnn"] = predicted_validation

In [ ]:
plt.rc('font', size=17)

plt.figure(num=1, figsize=(16, 12))

first_Taylor_diagram = True

list_colors = ["r", "k", "y", "b", "g", "m", "c", "darkorange", "lime", "paleturquoise", "gold"]
dict_label = {"observations": "c"}

data_to_use = data_validation

init_sdev, init_crmsd, init_ccoef = get_taylor_plot_stats(data_to_use, "rsds_stations", "rsds_stations")

list_columns_model = [
    "rsds_nora3",
    "rsds_era5",
    "SSI_value_cmsaf_sis",
    "rsds_sklearn",
    "rsds_fcnn",
]

for crrt_color, crrt_model in zip(list_colors[0:len(list_columns_model)], list_columns_model):
    dict_label[crrt_model] = crrt_color
    
    crrt_list_sdev = [init_sdev]
    crrt_list_crmsd = [init_crmsd]
    crrt_list_ccoef = [init_ccoef]
    
    list_stations_to_use = data_to_use.name.unique()
    
    for crrt_station in list_stations_to_use:
        
        pd_crrt_station = data_to_use[data_to_use["name"] == crrt_station]

        crrt_sdev, crrt_crmsd, crrt_ccoef = get_taylor_plot_stats(pd_crrt_station, "rsds_stations", crrt_model)
        crrt_list_sdev.append(crrt_sdev)
        crrt_list_crmsd.append(crrt_crmsd)
        crrt_list_ccoef.append(crrt_ccoef)
        
    crrt_sdev = np.array(crrt_list_sdev)
    crrt_crmsd = np.array(crrt_list_crmsd)
    crrt_ccoef = np.array(crrt_list_ccoef)

    time.sleep(1.0)
    
    if first_Taylor_diagram:
        sm.taylor_diagram(crrt_sdev, crrt_crmsd, crrt_ccoef, markercolor=crrt_color, markersize=4)
    else:
        try:
            sm.taylor_diagram(crrt_sdev, crrt_crmsd, crrt_ccoef, overlay = 'on', markercolor=crrt_color, markerLabel=dict_label, markersize=5)
        except:
            pass
    
    first_Taylor_diagram = False
    
    pd_to_aggregate = pd.DataFrame()
    
    for crrt_station in list_stations_to_use:
        pd_crrt_station = data_to_use[data_to_use["name"] == crrt_station]
        if len(pd_crrt_station) < 20:
            continue
        pd_to_aggregate = pd.concat([pd_to_aggregate, pd_crrt_station], ignore_index=True)
    
    # inspired from https://github.com/PeterRochford/SkillMetrics/blob/master/skill_metrics/taylor_diagram.py to put in the correct polar coordinates
    crrt_sdev, crrt_crmsd, crrt_ccoef = get_taylor_plot_stats(pd_to_aggregate, "rsds_stations", crrt_model)
    crrt_sdev = np.array([crrt_sdev])
    crrt_crmsd = np.array([crrt_crmsd])
    crrt_ccoef = np.array([crrt_ccoef])
    check_taylor_stats(crrt_sdev, crrt_crmsd, crrt_ccoef, 0.01)
    rho, theta = crrt_sdev, np.arccos(crrt_ccoef)
    
    plt.scatter(rho * np.cos(theta), rho * np.sin(theta), s=[512], c=[crrt_color], marker="X", label=crrt_model)
    
plt.scatter([1.0], [0.0], s=[512], c=["c"], label="truth")
plt.legend(bbox_to_anchor=(1.25, 1.25), loc='upper right')
plt.tight_layout()